In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
def load_doc(filename):
    file = open(filename , mode ='rt' , encoding = 'utf-8')
    text = file.read()
    file.close()
    return text

In [3]:
def to_pairs(text):
    lines = text.strip().split('\n') #1d list of strings
    eng_hin_pairs = [line.split('\t') for line in lines] #list
    eng_hin_pairs = np.delete(eng_hin_pairs , [0,1,2,3,4] , axis = 1)
#     eng_hin_pairs = np.delete(eng_hin_pairs , 0 , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    return eng_hin_pairs   

In [4]:
doc = load_doc('hindi-visual-genome-train.txt')
print(doc)

hin_eng_pairs = to_pairs(doc)

english_sentences = []
hindi_sentences = []
for i in range(len(hin_eng_pairs)):
    english_sentences.append(hin_eng_pairs[i][0])
    hindi_sentences.append(hin_eng_pairs[i][1])
len(hin_eng_pairs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



28930

In [5]:
hin_eng_pairs

[['it is an indoor scene', 'यह एक इनडोर दृश्य है'],
 ['Computer screens turned on', 'कंप्यूटर स्क्रीन चालू'],
 ['man has short hair', 'आदमी के छोटे बाल हैं'],
 ["photo album open on an adult's lap", 'एक वयस्क की गोद में फोटो एल्बम खुला'],
 ['there is a group of girls beside the black car',
  'काली कार के पास लड़कियों का एक समूह है'],
 ['Child in a stroller', 'एक घुमक्कड़ में बच्चा'],
 ['Tall metal lightpost', 'लंबा धातु प्रकाश पोस्ट'],
 ['wall is painted white', 'दीवार सफेद रंग की है'],
 ['there are several pictures on the wall', 'दीवार पर कई चित्र हैं'],
 ['woman facing the ocean', 'सागर का सामना कर रही महिला'],
 ['this is an office layout', 'यह एक कार्यालय लेआउट है'],
 ['four metallic chairs', 'चार धातु की कुर्सियाँ'],
 ['Clutter is on a table', 'क्लटर एक मेज पर है'],
 ['a white microwave oven', 'एक सफेद माइक्रोवेव ओवन'],
 ['White SUV driving through intersection',
  'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग'],
 ['Person crossing street with umbrella', 'छतरी के साथ सड़क पार करते व्य

In [6]:
len(hin_eng_pairs)

28930

In [7]:
print(type(hin_eng_pairs))

<class 'list'>


In [8]:
hindi_sentences

['यह एक इनडोर दृश्य है',
 'कंप्यूटर स्क्रीन चालू',
 'आदमी के छोटे बाल हैं',
 'एक वयस्क की गोद में फोटो एल्बम खुला',
 'काली कार के पास लड़कियों का एक समूह है',
 'एक घुमक्कड़ में बच्चा',
 'लंबा धातु प्रकाश पोस्ट',
 'दीवार सफेद रंग की है',
 'दीवार पर कई चित्र हैं',
 'सागर का सामना कर रही महिला',
 'यह एक कार्यालय लेआउट है',
 'चार धातु की कुर्सियाँ',
 'क्लटर एक मेज पर है',
 'एक सफेद माइक्रोवेव ओवन',
 'चौराहे के माध्यम से सफेद एसयूवी ड्राइविंग',
 'छतरी के साथ सड़क पार करते व्यक्ति',
 'इमारत पर झुकाव में आदमी ग्रे पैंट में',
 'भवन पर खिड़की',
 'कारों के बीच में खड़ा एक आदमी',
 'दीवार पर चित्रकला',
 'कुर्सी का पैर',
 'एक नाव की खिड़की',
 'कैबिनेट लकड़ी है',
 'एक आदमी एक किताब पढ़ रहा है',
 'पानी में तैरता हुआ व्यक्ति',
 'चमकती छाया के साथ दीपक',
 'प्लेट पर फल',
 'एक चीनी मिट्टी के बरतन सफ़ेद प्लेट',
 'काला खेल नियंत्रक',
 'कार की डिक्की में कुछ डालने वाला व्यक्ति।',
 'एक सड़क के साथ ग्रीन कार पार्क किया',
 'गली पर कार',
 'निर्माण कर्मचारी कवरलेस पहनते हैं',
 'पीले दरवाजे के साथ एक काली कार',
 '

In [9]:
def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    clean_eng_lines = []
    for line in lines:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [re_print.sub('' , w) for w in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        clean_eng_lines.append(line)
    return clean_eng_lines    

In [10]:
clean_eng_lines = clean_english_data(english_sentences)

# print(english_sentences)
print(english_sentences[3])
print(clean_eng_lines[3])

photo album open on an adult's lap
photo album open on an adults lap


In [11]:
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [12]:
def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"|",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

In [13]:
def clean_hindi_data(lines):
    clean_hin_lines = []
    for line in lines:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        clean_hin_lines.append(line)
    return clean_hin_lines  

In [14]:
clean_hindi_lines = clean_hindi_data(hindi_sentences)
print(hindi_sentences[133])
print(clean_hindi_lines[133])

तार से जुड़ा तार
तार से जुड़ा तार


In [15]:
len(clean_eng_lines)

28930

In [20]:
clean_hindi_lines.index('एक पेड़ की शाखा पर बैठे दो पक्षी')

23278

In [22]:
clean_hindi_lines.index('व्यक्ति पर नीला और सफेद निकर')

9480

In [23]:
input_train , input_val, target_train, target_val = train_test_split( clean_hindi_lines, clean_eng_lines , test_size = 0.2, random_state = 42)

### Adding SOS and EOS ,  PREPARING INPUTS FOR ENCODER AND DECODER

In [24]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

In [25]:
#input sentences
input_sentences = input_train

#output sentences
for line in target_train:
    line = line + ' <eos>'
    output_sentences.append(line)

#output sentence input

for line in target_train:
    line = '<sos> ' + line
    output_sentences_inputs.append(line)


In [26]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 23144
num samples output: 23144
num samples output input: 23144


In [27]:
BATCH_SIZE = 64
EPOCHS = 10
LSTM_NODES =100
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [28]:
# for hindi vocabulary
hin_lang_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
hin_lang_tokenizer.fit_on_texts(clean_hindi_lines)
input_integer_seq = hin_lang_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = hin_lang_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 7005
Length of longest sentence in input: 30


In [29]:
# for english vocabulary

eng = []
for i in clean_eng_lines:
    i = '<sos> ' + i + ' <eos>'
    eng.append(i)
len(eng)

eng_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
eng_lang_tokenizer.fit_on_texts(eng)
output_integer_seq = eng_lang_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = eng_lang_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = eng_lang_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 5553
Length of longest sentence in the output: 25


In [30]:
# pad encoder inputs and decoder inputs
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

encoder_input_sequences.shape: (23144, 30)
encoder_input_sequences[172]: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0 82 85 58]
decoder_input_sequences.shape: (23144, 25)
decoder_input_sequences[172]: [ 2 37 20 44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0]


### word embeddings

In [31]:
# english embeddings
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt' ,  encoding = 'utf-8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:] , dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [32]:
num_words = min(MAX_NUM_WORDS, len(word2idx_outputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_outputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [33]:
# hindi embeddings
embeddings_dictionary_h = dict()
glove_file = open('hi-d100-glove.txt' ,  encoding = 'UTF-8', errors='ignore')
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:] , dtype='float32')
    embeddings_dictionary_h[word] = vector_dimensions

glove_file.close()

In [34]:
len(embeddings_dictionary)

400000

In [35]:
num_words_h = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix_h = zeros((num_words_h, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

In [36]:
num_words

5554

In [37]:
num_words_h

7006

# MODEL

In [38]:
## CREATING THE MODEL

In [39]:
decoder_targets_one_hot = np.zeros((len(input_sentences), max_out_len, num_words_output),dtype='float32')

In [40]:
decoder_targets_one_hot.shape
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [41]:
num_words

5554

In [42]:
num_words_h

7006

## create the encoder and decoders

In [43]:
embedding_layer_h = Embedding(num_words_h, EMBEDDING_SIZE, weights=[embedding_matrix_h], input_length=max_input_len)
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer_h(encoder_inputs_placeholder)
encoder = LSTM(100, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [44]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_out_len)

decoder_inputs_placeholder = Input(shape=(max_out_len,))
decoder_inputs_x = embedding_layer(decoder_inputs_placeholder)

decoder_lstm = LSTM(100 , return_sequences = True , return_state = True)

decoder_outputs , _ , _ = decoder_lstm(decoder_inputs_x , initial_state = encoder_states)

In [45]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## compile the model:

In [46]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder] , decoder_outputs)

model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [47]:
# !pip install pydot
# !pip install graphviz

In [48]:
# plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [49]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_split=0.1,
)

Epoch 1/10
326/326 [==============================] - 102s 314ms/step - loss: 1.5173 - accuracy: 0.8013 - val_loss: 1.0185 - val_accuracy: 0.8433
Epoch 2/10
326/326 [==============================] - 103s 317ms/step - loss: 0.9013 - accuracy: 0.8592 - val_loss: 0.8381 - val_accuracy: 0.8682
Epoch 3/10
326/326 [==============================] - 103s 316ms/step - loss: 0.7583 - accuracy: 0.8767 - val_loss: 0.7323 - val_accuracy: 0.8822
Epoch 4/10
326/326 [==============================] - 104s 319ms/step - loss: 0.6571 - accuracy: 0.8913 - val_loss: 0.6529 - val_accuracy: 0.8953
Epoch 5/10
326/326 [==============================] - 104s 318ms/step - loss: 0.5861 - accuracy: 0.9022 - val_loss: 0.6017 - val_accuracy: 0.9033
Epoch 6/10
326/326 [==============================] - 104s 319ms/step - loss: 0.5365 - accuracy: 0.9103 - val_loss: 0.5677 - val_accuracy: 0.9089
Epoch 7/10
326/326 [==============================] - 104s 319ms/step - loss: 0.4968 - accuracy: 0.9165 - val_loss: 0.5386 -

In [ ]:
# model.save('model_1_hin-eng_1')

In [50]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      700600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 100)      555400      input_2[0][0]                    
______________________________________________________________________________________________

In [51]:
# new_model = tf.keras.models.load_model('model_1_hin-eng')

# # Check its architecture
# new_model.summary()

In [52]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [53]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [54]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = embedding_layer(decoder_inputs_single)

In [55]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [56]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [57]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [58]:
# from keras.utils import plot_model
# plot_model(decoder_model, to_file='model_plot_dec.png', show_shapes=True, show_layer_names=True)

In [59]:
### predictions

In [60]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [61]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [64]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: एक लैपटॉप कंप्यूटर पर बिल्ली सोई है
Response: cat is sitting on a laptop


In [85]:
test_sentences = input_val[0:300]
test_sentences_english = target_val[0:300]

In [67]:
# test_sentences.index('व्यक्ति पर नीला और सफेद निकर |')

ValueError: 'व्यक्ति पर नीला और सफेद निकर |' is not in list

In [86]:
print(len(test_sentences) , len(test_sentences_english))

300 300


In [87]:
test_sentences.index('')

ValueError: '' is not in list

In [70]:
test_sentences.pop(1346) , test_sentences_english.pop(1346)

('', 'tabby cat resting under desk')

In [71]:
print(len(test_sentences) , len(test_sentences_english))

5785 5785


In [72]:
test_sentences.index('')

1655

In [73]:
test_sentences.pop(1655) , test_sentences_english.pop(1655)

('', 'tall giraffe standing in dirt')

In [74]:
test_sentences.index('')

4508

In [75]:
test_sentences.pop(4508) , test_sentences_english.pop(4508)

('', 'the ramp says supreme')

In [77]:
# test_sentences.index('')

In [78]:
test_sentences.pop(258)
test_sentences_english.pop(258)
print(len(test_sentences) , len(test_sentences_english))

5782 5782


In [80]:
# test_sentences.index('एक पेड़ की शाखा पर बैठे दो पक्षी ।')

In [81]:
# test_sentences.pop(118)
# test_sentences_english.pop(118)
# print(len(test_sentences) , len(test_sentences_english))

5781 5781


In [ ]:
# test_sentences.pop(247)
# test_sentences_english.pop(247)
# print(len(test_sentences) , len(test_sentences_english))

In [ ]:
# test_sentences.pop(280)
# test_sentences_english.pop(280)
# print(len(test_sentences) , len(test_sentences_english))

In [ ]:
# t1 = []
# for i in test_sentences:
#     t = clean_text(i)
#     t1.append(t)    
        

In [83]:
test_sentences

['यह आकाश है',
 'बैंगनी और सफेद जूते के साथ लड़का',
 'डैपर लाल टोपी पहने हुए आदमी',
 'हरे रंग के पत्ते',
 'एक धावक पहले बेस से आगे निकल गया',
 'एक लाल रंग की ट्रेन',
 'अंधेरे लाल इमारत पर विचलित',
 'आदमी पर काला सूट जैकेट',
 'बड़ा धूसर भारतीय हाथी',
 'आदमी के पास एक अंगूठी है',
 'लड़की ने हार पहना हुआ है',
 'बोरहोल से पानी खींचता आदमी',
 'सड़क संकेत',
 'इमारत पर खिडकि',
 'मैदान में बैंगनी फूलों के शीर्ष',
 'फोन पकड़े हुए एक महीला',
 'सफेद प्लग के साथ काले आइपॉड',
 'ज़ेबरा पर धारियाँ',
 'काले जैतून का एक कटा हुआ टुकड़ा',
 'गाय लेटी हुई थी',
 'लकड़ी के स्लेट के साथ लकड़ी की बेंच',
 'यह एक इमारत है',
 'महिलाएं कुर्सी पर बैठी हैं',
 'प्लेट पर कटा हुआ लाल टमाटर',
 'दो जर्सी पर लिखा गया है',
 'यह एक शौचालय धनुष है',
 'हाइड्रेंट पीला है',
 'एक मेज पर बैठे लोग',
 'ब्लू सर्फ बोर्ड',
 'जिराफ के पास धातु का पिंजरा',
 'पानी का उथला शरीर',
 'लोगों का एक समूह',
 'टेबल पर फोन',
 'नाव पर कांच की खिड़की',
 'रेलगाड़ी पर ट्रेन',
 'एक मेज पर एक टीव',
 'एक चमकदार नीली कॉफी मग',
 'बाथरूम में दो महिलाएं',
 'ट

In [88]:
test_tensor = [[hin_lang_tokenizer.word_index[s] for s in hi.split(' ')] for hi in test_sentences[0:300]]
# print(max_length(test_tensor))
#pad
test_sequences = pad_sequences(test_tensor, maxlen=max_input_len, padding='pre')
print(max_input_len)
# encoder_input_sequences = pad_sequences(input_tensor, maxlen=max_length_inp,padding='pre')

30


In [89]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences[0:200]))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    hindi_test = test_sentences_hindi[i : i+1]
    print('-')
    print('Input:', test_sentences[i])
    print('Response:', translation)

    print('Original Hindi Sentence:' , hindi_test[0])
    print(type(hindi_test[0]))

    


NameError: name 'test_sentences_hindi' is not defined

In [92]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    english_test = test_sentences_english[i : i+1]

    english_ref = english_test[0]
    english_ref = english_ref.split()
#     print(hindi_ref)
    candidate = translation.split()
#     print(candidate)
    score = sentence_bleu([english_ref], candidate, weights=(1, 0, 0, 0) )

#     print(score)
    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)
    

C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

100
the average bleu score is  0.6227247479333181


In [94]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    english_test = test_sentences_english[i : i+1]

    english_ref = english_test[0]
    english_ref = english_ref.split()
#     print(hindi_ref)
    candidate = translation.split()
#     print(candidate)
    score = sentence_bleu([english_ref], candidate, weights=(0, 1, 0, 0) )

#     print(score)
    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.46696628938835716


In [95]:
# i = np.random.choice(len(test_sentences))
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_bleu
count = 0
bleu_score = 0
for j in range (0,100):
    i = np.random.choice(len(test_sentences))
    input_seq = test_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    english_test = test_sentences_english[i : i+1]

    english_ref = english_test[0]
    english_ref = english_ref.split()
#     print(hindi_ref)
    candidate = translation.split()
#     print(candidate)
    score = sentence_bleu([english_ref], candidate, weights=(0.25, 0.25, 0.25, 0.25) )

#     print(score)
    count = count + 1
    bleu_score = bleu_score + score
    
avg_bleu_score = bleu_score/count
print(count)
print("the average bleu score is ", avg_bleu_score)

100
the average bleu score is  0.22313088685509824
